In [31]:
!pip install torch
!pip install torchvision
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np
from torchvision import datasets, transforms
import torch.nn.functional as F

from AgeDatasetLoader import AgeDataset

In [32]:
USE_GPU = True
num_class = 100
dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cpu


In [ ]:
age_train = AgeDataset("age_data/age-train.csv")
loader_train = DataLoader(age_train, batch_size=32, shuffle=True, num_workers=2)

age_val = AgeDataset("age_data/age-val.csv")
loader_val = DataLoader(age_val, batch_size=32, shuffle=True, num_workers=2)


age_test = AgeDataset("age_data/age-test.csv")
loader_test = DataLoader(age_test, batch_size=32, shuffle=True, num_workers=2)



In [ ]:
def check_accuracy_part34(loader, model):
    print("iniating check acc func")
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
    return acc

In [ ]:


def train_part34(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: The accuracy of the model
    """
    print("iniating train func")
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            
            
            print(t)
          #  print(scores.size())
          #  print(x.size(), y.size())
            loss = F.cross_entropy(scores, y)
            

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()
      
            if (t + 1) % print_every == 0:
                print('Epoch %d, Iteration %d, loss = %.4f' % (e, t + 1, loss.item()))
                check_accuracy_part34(loader_val, model)
                print()
    return check_accuracy_part34(loader_val, model)

In [ ]:
class VGG_16(nn.Module):
    def __init__(self):
        super(VGG_16, self).__init__()
        
        self.conv1_1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
        self.bn1_1 = nn.BatchNorm2d(64)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.bn1_2 = nn.BatchNorm2d(64)
        self.max1 = nn.MaxPool2d((2,2), stride=2, padding=1)
        
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2_1 = nn.BatchNorm2d(128)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn2_2 = nn.BatchNorm2d(128)
        self.max2 = nn.MaxPool2d((2, 2), stride=2, padding=1)


        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3_1 = nn.BatchNorm2d(256)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.bn3_2 = nn.BatchNorm2d(256)

        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4_1 = nn.BatchNorm2d(512)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn4_2 = nn.BatchNorm2d(512)

        self.max3 = nn.MaxPool2d((2, 2), stride=2, padding=1)

        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn5_1 = nn.BatchNorm2d(512)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn5_2 = nn.BatchNorm2d(512)

        self.fc1 = nn.Linear(25088 , 512)
        self.fc2 = nn.Linear(512, 24)
        self.fc3 = nn.Linear(24, 24)  # 100 classes for CIFAR-100
        self.dropout = nn.Dropout(0.5)
        self.relu =  nn.ReLU(inplace=True)
   


  

    def forward(self,x):

         x = self.conv1_1(x)
         x = self.bn1_1(x)
         x = self.relu(x)
         x = self.conv1_2(x)
         x = self.bn1_2(x)
         x = self.relu(x)
         x = self.max1(x)


         x = self.conv2_1(x)
         x = self.bn2_1(x)
         x = self.relu(x)
         x = self.conv2_2(x)
         x = self.bn2_2(x)
         x = self.relu(x)
         x = self.max2(x)


         x = self.conv3_1(x)
         x = self.bn3_1(x)
         x = self.relu(x)
         x = self.conv3_2(x)
         x = self.bn3_2(x)
         x = self.relu(x)



         x = self.conv4_1(x)
         x = self.bn4_1(x)
         x = self.relu(x)
         x = self.conv4_2(x)
         x = self.bn4_2(x)
         x = self.relu(x)
         x = self.max3(x)


         x = self.conv5_1(x)
         x = self.bn5_1(x)
         x = self.relu(x)
         x = self.conv5_2(x)
         x = self.bn5_2(x)

         x = x.view(x.size(0), -1)
         x = self.fc1(x)
         x = self.fc2(x)
         x = self.fc3(x)
        

         return x

In [ ]:

model = VGG_16();
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


print_every = 700
train_part34(model, optimizer, epochs=10)
print_every = 100